# AI Image Classifier
- **TensorFlow:** to use a pretrained machine learning model that can look at an image and classify it
- **OpenCV:** to allow manipulations on the image so it can be passed to TensorFlow